In [10]:
require 'cutorch' -- Use CUDA if available
require 'cudnn'
require 'cunn'

-- print(cudnn)

-- Load dependencies
optim = require 'optim'
gnuplot = require 'gnuplot'
image = require 'image'
nn = require 'nn'

require 'paths'

count = cutorch.getDeviceCount()
--     torch.setdefaulttensortype('torch.CudaTensor')    
print('GPU is ON')
for i = 1, count do
    print('Device ' .. i .. ':')
    freeMemory, totalMemory = cutorch.getMemoryUsage(i)
    print('\t Free memory ' .. freeMemory)
    print('\t Total memory ' .. totalMemory)
end

cutorch.setDevice(2)
torch.setnumthreads(12)

-- Set up Torch
print('Setting up')
torch.setdefaulttensortype('torch.FloatTensor')
torch.manualSeed(1) 
cutorch.manualSeed(torch.random())
-- end

cuda = true
hasCudnn = true

print('Has cudnn: ')
print(hasCudnn)


package.loaded['./modelTools'] = nil
DataProvider = require ('./modelTools')

package.loaded['./DataProvider2D'] = nil
DataProvider = require ('./DataProvider2D')

package.loaded['setupAAEGANv2'] = nil
setup = require 'setupAAEGANv2'

model_opts = setup.getModelOpts()

model_opts.parent_dir = './trainAAE2D_256'
model_opts.model_name = 'caae-nopool_GAN_v2_AAEGANv1_4_alldat'
model_opts.image_dir = '/root/images/release_4_1_17/release_v2/aligned/2D/'

model_opts.save_dir = model_opts.parent_dir .. '/' .. model_opts.model_name

model_opts.channel_inds_in = torch.LongTensor{1,3}
model_opts.channel_inds_out = torch.LongTensor{1,3}
model_opts.nChOut = 3
model_opts.nChIn = 3
model_opts.rotate = false

dataProvider = DataProvider.create(model_opts.image_dir, model_opts)

print('Loading Models')

encoder = torch.load(model_opts.save_dir .. '/encoder.t7')

decoder:cuda()
encoder:cuda()

print('Done Loading Models')

decoder:evaluate()
encoder:evaluate()

package.loaded['utils'] = nil
setup = require 'utils'

print(model_opts.image_dir)

image_dir = model_opts.image_dir
image_paths = {}
classes = {}
c = 0
for dir in paths.iterdirs(image_dir) do
    print('Loading images from ' .. image_dir .. '/' .. dir)

    im_dir = image_dir .. '/' .. dir .. '/'
    im_pattern = 'png'
    
    local p = {}
    local c_files = 1
    
    for f in paths.files(im_dir, im_pattern) do
        p[c_files] = im_dir .. f
        c_files = c_files+1
    end
    
    -- natural sort
    local image_paths_tmp = utils.alphanumsort(p)
    
    
    for i = 1,#image_paths_tmp do
        c = c+1

        image_paths[c] = image_paths_tmp[i]

        tokens = utils.split(image_paths_tmp[i], '/')
        classes[c] = tokens[#tokens-1]    
    end
end

-- print(classes)

classes, labels = utils.unique(classes)

print(classes)



save_dir = model_opts.save_dir .. '/' .. 'gridded'
paths.mkdir(save_dir)

nsteps = 21;

pos = torch.linspace(-3, 3, nsteps);

im_out = {}

-- for k = 1, 10 do
    imgs_i = {}
    for i = 1,nsteps do
        imgs_j = {}
        for j = 1,nsteps do
            dat = {}
        
            dat[1] = torch.zeros(2,model_opts.nLatentDims):cuda()
            dat[1][{{},3}] = pos[i]
            dat[1][{{},4}] = pos[j]

            imgs_j[j] = decoder:forward(dat)[1]:float()
            
        end
        
        imgs_i[i] = torch.cat(imgs_j, 2)
    end
    
    imgs_struct = torch.cat(imgs_i,3)

    imgs_out = torch.zeros(3, imgs_struct:size(2), imgs_struct:size(2))

    imgs_out[1] = imgs_struct[1]
    imgs_out[3] = imgs_struct[2]
    
    save_path = save_dir .. '/cellnuc.png'
    image.save(save_path, imgs_out)
-- end

encoder = nil




GPU is ON	
Device 1:	


	 Free memory 11500444160	
	 Total memory 12782075904	
Device 2:	


	 Free memory 11997492224	
	 Total memory 12778799104	
Device 3:	
	 Free memory 12176064512	
	 Total memory 12782075904	
Setting up	
Has cudnn: 	
true	


Loading data from ./trainAAE2D_256	


Done	
Loading Models	


...

Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Alpha tubulin	


Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Desmoplakin	


Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Myosin IIB	


Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Fibrillarin	


{
  1 : Beta actin
  2 : ZO1
  3 : Sec61 beta
  4 : Alpha actinin
  5 : Tom20
  6 : Lamin B1
  7 : Alpha tubulin
  8 : Desmoplakin
  9 : Myosin IIB
  10 : Fibrillarin
}


In [7]:
save_dir

./trainAAE2D_256/caae-nopool_GAN_v2_AAEGANv1_4_alldat/gridded_ref	


In [1]:
require 'cutorch' -- Use CUDA if available
require 'cudnn'
require 'cunn'

-- print(cudnn)

-- Load dependencies
optim = require 'optim'
gnuplot = require 'gnuplot'
image = require 'image'
nn = require 'nn'

require 'paths'

count = cutorch.getDeviceCount()
--     torch.setdefaulttensortype('torch.CudaTensor')    
print('GPU is ON')
for i = 1, count do
    print('Device ' .. i .. ':')
    freeMemory, totalMemory = cutorch.getMemoryUsage(i)
    print('\t Free memory ' .. freeMemory)
    print('\t Total memory ' .. totalMemory)
end

cutorch.setDevice(2)
torch.setnumthreads(12)

-- Set up Torch
print('Setting up')
torch.setdefaulttensortype('torch.FloatTensor')
torch.manualSeed(1) 
cutorch.manualSeed(torch.random())
-- end

cuda = true
hasCudnn = true

print('Has cudnn: ')
print(hasCudnn)


package.loaded['./modelTools'] = nil
DataProvider = require ('./modelTools')

package.loaded['./DataProvider2D'] = nil
DataProvider = require ('./DataProvider2D')

package.loaded['setupAAEGANv2'] = nil
setup = require 'setupAAEGANv2'

model_opts = setup.getModelOpts()

model_opts.parent_dir = './trainAAE2D_256'
model_opts.model_name = 'caae-nopool_GAN_v2_AAEGANv1_4_alldat_pattern'
model_opts.image_dir = '/root/images/release_4_1_17/release_v2/aligned/2D/'

model_opts.save_dir = model_opts.parent_dir .. '/' .. model_opts.model_name

model_opts.channel_inds_in = torch.LongTensor{1,2,3}
model_opts.channel_inds_out = torch.LongTensor{1,2,3}
model_opts.nChOut = 3
model_opts.nChIn = 3
model_opts.rotate = false

dataProvider = DataProvider.create(model_opts.image_dir, model_opts)

print('Loading Models')

decoder = torch.load(model_opts.save_dir .. '/decoder.t7')
encoder = torch.load(model_opts.save_dir .. '/encoder.t7')

decoder:cuda()
encoder:cuda()

print('Done Loading Models')

decoder:evaluate()
encoder:evaluate()

package.loaded['utils'] = nil
setup = require 'utils'

print(model_opts.image_dir)

image_dir = model_opts.image_dir
image_paths = {}
classes = {}
c = 0
for dir in paths.iterdirs(image_dir) do
    print('Loading images from ' .. image_dir .. '/' .. dir)

    im_dir = image_dir .. '/' .. dir .. '/'
    im_pattern = 'png'
    
    local p = {}
    local c_files = 1
    
    for f in paths.files(im_dir, im_pattern) do
        p[c_files] = im_dir .. f
        c_files = c_files+1
    end
    
    -- natural sort
    local image_paths_tmp = utils.alphanumsort(p)
    
    
    for i = 1,#image_paths_tmp do
        c = c+1

        image_paths[c] = image_paths_tmp[i]

        tokens = utils.split(image_paths_tmp[i], '/')
        classes[c] = tokens[#tokens-1]    
    end
end

-- print(classes)

classes, labels = utils.unique(classes)

print(classes)




GPU is ON	
Device 1:	


	 Free memory 12096372736	
	 Total memory 12782075904	
Device 2:	


	 Free memory 12077301760	
	 Total memory 12778799104	
Device 3:	


	 Free memory 12176064512	
	 Total memory 12782075904	
Setting up	
Has cudnn: 	
true	


Loading data from ./trainAAE2D_256	


...

Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Alpha tubulin	


Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Desmoplakin	


Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Myosin IIB	


Loading images from /root/images/release_4_1_17/release_v2/aligned/2D//Fibrillarin	


{
  1 : Beta actin
  2 : ZO1
  3 : Sec61 beta
  4 : Alpha actinin
  5 : Tom20
  6 : Lamin B1
  7 : Alpha tubulin
  8 : Desmoplakin
  9 : Myosin IIB
  10 : Fibrillarin
}


In [ ]:

save_dir = model_opts.save_dir .. '/' .. 'gridded'
paths.mkdir(save_dir)

nsteps = 21;

pos = torch.linspace(-3, 3, nsteps);

im_out = {}

for k = 1, 10 do
    imgs_i = {}
    for i = 1,nsteps do
        imgs_j = {}
        for j = 1,nsteps do
            dat = {}
            dat[1] = torch.zeros(2,10):fill(-26):cuda()
            dat[1][{{},k}] = 1
        
            dat[2] = torch.zeros(2,model_opts.nLatentDims):cuda()
            dat[2][{{},3}] = pos[i]
            dat[2][{{},4}] = pos[j]
            
            dat[3] = torch.zeros(2,model_opts.nLatentDims):cuda()
            
            imgs_j[j] = decoder:forward(dat)[1]:float()
            
        end
        
        imgs_i[i] = torch.cat(imgs_j, 2)
    end
    
    imgs_struct = torch.cat(imgs_i,3)
--     itorch.image(imgs_struct)
    save_path = save_dir .. '/' .. classes[k] .. '.png'
    image.save(save_path, imgs_struct)
end


In [ ]:



print_images = function(encoder, decoder, dataProvider, train_or_test, save_dir)
        
    require 'paths'

    paths.mkdir(save_dir)

    ndat = dataProvider[train_or_test].labels:size(1)
    nclasses = dataProvider[train_or_test].labels:size(2)

    for j = 1,ndat do
--         print('Printing image ' .. j)
        img_num = torch.LongTensor{j}

        local label = dataProvider:getLabels(torch.LongTensor{j,j}, train_or_test)
        local img = dataProvider:getImages(torch.LongTensor{j,j}, train_or_test):cuda()
        local imsize = img:size()
        local out_latent = nil
        out_latent = encoder:forward(img)

        out_latent[3] = torch.zeros(2,model_opts.nLatentDims):cuda()

        local out_img = torch.Tensor(nclasses, 3, imsize[3], imsize[4]):cuda()

        im_path = dataProvider.image_paths[dataProvider[train_or_test].inds[j]]
        tokens = utils.split(im_path, '/|.')
        im_class = tokens[7]
        im_id = tokens[8]
        
        print('Printing images for ' .. im_class .. ' ' .. im_id)
        
        save_path = save_dir .. '/' .. im_class .. '_'.. im_id .. '_orig.png'
        image.save(save_path, img[1])
        
        for i = 1,#dataProvider.classes do
            save_path = save_dir .. '/' .. im_class .. '_'.. im_id .. '_pred_' .. classes[i] .. '.png'
            
            if not paths.filep(save_path) then
                one_hot = torch.ones(2,nclasses):fill(-25):cuda()
                one_hot[{{1},{i}}] = 0

                out_latent[1] = one_hot
                out_img = decoder:forward(out_latent)[1][2]


                image.save(save_path, out_img)
            end
            
        end
    end
end

print_images(encoder, decoder, dataProvider, 'train', model_opts.save_dir .. '/' .. 'pred_train')
print_images(encoder, decoder, dataProvider, 'test', model_opts.save_dir .. '/' .. 'pred_test')

In [28]:

classes_sort = {}
for i = 1,#classes do
    classes_sort[i] = classes[i]
end
    
-- print(classes_sort)
utils.alphanumsort(classes_sort)

order = torch.zeros(#classes)
order_bak = torch.zeros(#classes)
for i = 1, #classes do
    for j = 1, #classes do
        if classes_sort[i] == classes[j] then
            order[j] = i
            order_bak[i] = j
        end
    end
end
print(classes_sort)
-- print(classes)
-- print(order)


eval_images = function(encoder, dataProvider, train_or_test)
        
    require 'paths'


    ndat = dataProvider[train_or_test].labels:size(1)
    nclasses = dataProvider[train_or_test].labels:size(2)

    confmat = torch.zeros(nclasses, nclasses)
    
    for j = 1,ndat do
--         print('Printing image ' .. j)
        img_num = torch.LongTensor{j}

        label = dataProvider:getLabels(torch.LongTensor{j,j}, train_or_test)
        local img = dataProvider:getImages(torch.LongTensor{j,j}, train_or_test):cuda()
        local imsize = img:size()
--         local out_latent = nil
        out_latent = encoder:forward(img)        
        out_pred = out_latent[1][1]:type('torch.FloatTensor')

        tmp, pred_label = torch.max(out_pred,1)
        
        tmp, label = torch.max(label[1],1)

        confmat[order[label[1]]][order[pred_label[1]]] = confmat[order[label[1]]][order[pred_label[1]]] + 1
    end
end

eval_images(encoder, dataProvider, 'test')
print(confmat)


print('total dat: ' .. labels:size(1))

print('')
print('By label:')

labels_test = labels:index(1,dataProvider.test.inds)
labels_train = labels:index(1,dataProvider.train.inds)

for i = 1, #dataProvider.classes do
    i = order_bak[i]
    ntot = torch.sum(torch.eq(labels,i))
    ntest = torch.sum(torch.eq(labels_test,i))
    ntrain = torch.sum(torch.eq(labels_train,i))
    print(classes[i] .. ': ' .. ntot .. ', ' .. ntrain .. ', ' .. ntest)
end

{
  1 : Alpha actinin
  2 : Alpha tubulin
  3 : Beta actin
  4 : Desmoplakin
  5 : Fibrillarin
  6 : Lamin B1
  7 : Myosin IIB
  8 : Sec61 beta
  9 : Tom20
  10 : ZO1
}


 22   2   5   1   0   0   0   0   0   1
  0  36   3   0   0   0   0   1   1   0
  3   7  19   0   0   0   0   0   0   0
  1   0   1   7   0   0   0   0   0   1
  0   0   0   0  35   0   0   0   0   0
  0   0   0   0   0  46   0   0   0   0
  2   0   0   1   0   0   0   0   1   4
  0   1   0   0   0   0   0  50   0   0
  0   1   0   0   0   0   0   0  47   0
  1   0   0   1   0   0   2   0   0   1
[torch.FloatTensor of size 10x10]

total dat: 6077	
	
By label:	
Alpha actinin: 493, 462, 31	
Alpha tubulin: 1043, 1002, 41	


Beta actin: 542, 513, 29	
Desmoplakin: 229, 219, 10	
Fibrillarin: 988, 953, 35	
Lamin B1: 785, 739, 46	
Myosin IIB: 157, 149, 8	
Sec61 beta: 835, 784, 51	
Tom20: 771, 723, 48	
ZO1: 234, 229, 5	


In [34]:
-- latex formatted table shennanary

for i = 1,#classes do
    str = classes_sort[i]
    for j =1,#classes do
        str = str .. ' & ' .. confmat[i][j]
    end
    str = str .. ' \\\\'
    
    print(str)
end

Alpha actinin & 22 & 2 & 5 & 1 & 0 & 0 & 0 & 0 & 0 & 1 \\	
Alpha tubulin & 0 & 36 & 3 & 0 & 0 & 0 & 0 & 1 & 1 & 0 \\	
Beta actin & 3 & 7 & 19 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\	
Desmoplakin & 1 & 0 & 1 & 7 & 0 & 0 & 0 & 0 & 0 & 1 \\	
Fibrillarin & 0 & 0 & 0 & 0 & 35 & 0 & 0 & 0 & 0 & 0 \\	
Lamin B1 & 0 & 0 & 0 & 0 & 0 & 46 & 0 & 0 & 0 & 0 \\	
Myosin IIB & 2 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 4 \\	
Sec61 beta & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 50 & 0 & 0 \\	
Tom20 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 47 & 0 \\	
ZO1 & 1 & 0 & 0 & 1 & 0 & 0 & 2 & 0 & 0 & 1 \\	


In [36]:
for i = 1, #dataProvider.classes do
    i = order_bak[i]
    ntot = torch.sum(torch.eq(labels,i))
    ntest = torch.sum(torch.eq(labels_test,i))
    ntrain = torch.sum(torch.eq(labels_train,i))
    print(classes[i] .. ' & ' .. ntot .. ' & ' .. ntrain .. ' & ' .. ntest .. ' \\\\')
end

Alpha actinin & 493 & 462 & 31 \\	
Alpha tubulin & 1043 & 1002 & 41 \\	
Beta actin & 542 & 513 & 29 \\	
Desmoplakin & 229 & 219 & 10 \\	
Fibrillarin & 988 & 953 & 35 \\	
Lamin B1 & 785 & 739 & 46 \\	
Myosin IIB & 157 & 149 & 8 \\	
Sec61 beta & 835 & 784 & 51 \\	


Tom20 & 771 & 723 & 48 \\	
ZO1 & 234 & 229 & 5 \\	


In [4]:
print('total dat: ' .. labels:size(1))

print('')
print('By label:')

labels_test = labels:index(1,dataProvider.test.inds)
labels_train = labels:index(1,dataProvider.train.inds)

for i = 1, #dataProvider.classes do
    ntot = torch.sum(torch.eq(labels,i))
    ntest = torch.sum(torch.eq(labels_test,i))
    ntrain = torch.sum(torch.eq(labels_train,i))
    print(classes[i] .. ': ' .. ntot .. ', ' .. ntrain .. ', ' .. ntest)
end

total dat: 6077	
	
By label:	
Beta actin: 542, 513, 29	
ZO1: 234, 229, 5	
Sec61 beta: 835, 784, 51	
Alpha actinin: 493, 462, 31	
Tom20: 771, 723, 48	
Lamin B1: 785, 739, 46	
Alpha tubulin: 1043, 1002, 41	


Desmoplakin: 229, 219, 10	
Myosin IIB: 157, 149, 8	
Fibrillarin: 988, 953, 35	


{
  1 : Beta actin
  2 : ZO1
  3 : Sec61 beta
  4 : Alpha actinin
  5 : Tom20
  6 : Lamin B1
  7 : Alpha tubulin
  8 : Desmoplakin
  9 : Myosin IIB
  10 : Fibrillarin
}
{
  1 : Alpha actinin
  2 : Alpha tubulin
  3 : Beta actin
  4 : Desmoplakin
  5 : Fibrillarin
  6 : Lamin B1
  7 : Myosin IIB
  8 : Sec61 beta
  9 : Tom20
  10 : ZO1
}
{
  1 : Alpha actinin
  2 : Alpha tubulin
  3 : Beta actin
  4 : Desmoplakin
  5 : Fibrillarin
  6 : Lamin B1
  7 : Myosin IIB
  8 : Sec61 beta
  9 : Tom20
  

10 : ZO1
}
{
  1 : Beta actin
  2 : ZO1
  3 : Sec61 beta
  4 : Alpha actinin
  5 : Tom20
  6 : Lamin B1
  7 : Alpha tubulin
  8 : Desmoplakin
  9 : Myosin IIB
  10 : Fibrillarin
}
  4
  7
  1
  8
 10
  6
  9
  3
  5
  2
[torch.FloatTensor of size 10]



ZO1	
